In [224]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [225]:
from pearce.emulator import OriginalRecipe, ExtraCrispy
from pearce.mocks import cat_dict
import numpy as np
from os import path

In [226]:
import tensorflow as tf

In [227]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [228]:
training_file = '/home/sean/PearceRedMagicXiChinchilla.hdf5'
test_file = '/home/sean/PearceRedMagicXiChinchillaTest.hdf5'

em_method = 'nn'
split_method = 'random'

In [229]:
a = 1.0
z = 1.0/a - 1.0

In [230]:
fixed_params = {'z':z}

In [231]:
emu = OriginalRecipe(training_file, method = em_method, fixed_params=fixed_params,
                    hyperparams = {'hidden_layer_sizes': (10),
                                 'activation': 'relu', 'verbose': True, 
                                    'tol': 1e-8, 'learning_rate_init':1e-4,\
                                   'max_iter':10, 'alpha':0, 'early_stopping':False, 'validation_fraction':0.3})

Iteration 1, loss = 1.19109137
Iteration 2, loss = 1.09983267
Iteration 3, loss = 1.01913283
Iteration 4, loss = 0.94804537
Iteration 5, loss = 0.88493168
Iteration 6, loss = 0.82855000
Iteration 7, loss = 0.77787011
Iteration 8, loss = 0.73204788
Iteration 9, loss = 0.69042516
Iteration 10, loss = 0.65217603


In [242]:
x_train, y_train,yerr_train = emu.x, emu.y, emu.yerr

y_train = y_train*(emu._y_std + 1e-5) + emu._y_mean
yerr_train = yerr_train*(emu._y_std+1e-5)

In [243]:
x_test, y_test, ycov_test, _ = emu.get_data(test_file, fixed_params, None, False)
x_test = (x_test - emu._x_mean)/(emu._x_std+1e-5)

split_ycov = np.dsplit(ycov_test, ycov_test.shape[-1])
        #fullcov = block_diag(*[yc[:,:,0] for yc in split_ycov])
yerr_test = np.sqrt(np.hstack(np.diag(syc[:,:,0]) for syc in split_ycov))

In [244]:
def n_layer_fc(x, hidden_sizes, training=False, l = 1e-8):
    initializer = tf.variance_scaling_initializer(scale=2.0)
    regularizer = tf.contrib.layers.l2_regularizer(l)
    fc_output = tf.layers.dense(x, hidden_sizes[0], activation=tf.nn.relu,
                                 kernel_initializer = initializer, kernel_regularizer = regularizer)
                                 #kernel_regularizer = tf.nn.l2_loss)
    #fc2_output = tf.layers.dense(fc1_output, hidden_sizes[1], activation=tf.nn.relu,
    #                             kernel_initializer = initializer, kernel_regularizer = regularizer)
    for size in hidden_sizes[1:]:
        fc_output = tf.layers.dense(fc_output, size, activation=tf.nn.relu, kernel_initializer=initializer,
                                 kernel_regularizer = regularizer)
    pred = tf.layers.dense(fc_output, 1, kernel_initializer=initializer, 
                              kernel_regularizer = regularizer)[:,0]#,
    return pred

In [245]:
def n_layer_fc_do_bn(x, hidden_sizes, training=False, l = 1e-7):
    initializer = tf.variance_scaling_initializer(scale=2.0)
    regularizer = tf.contrib.layers.l2_regularizer(l)
    nl_out = x

    for size in hidden_sizes:
        fc_output = tf.layers.dense(nl_out, size,
                                 kernel_initializer = initializer, kernel_regularizer = regularizer)
        #bd_out = tf.layers.dropout(fc_output, 0.5, training = training)
        #bn_out = tf.layers.batch_normalization(fc_output, axis = -1, training=training)
        nl_out = tf.nn.relu(fc_output)#tf.nn.leaky_relu(bn_out, alpha=0.01)
    
    pred = tf.layers.dense(nl_out, 1, kernel_initializer=initializer, 
                              kernel_regularizer = regularizer)[:,0]#,
    return pred

In [246]:
def optimizer_init_fn(learning_rate = 1e-5):
    return tf.train.AdamOptimizer(learning_rate)

In [247]:
from sklearn.metrics import r2_score, mean_squared_error

In [248]:
def check_accuracy(sess, val_data,batch_size, x, weights, preds, is_training=None):
    val_x, val_y, val_yerr = val_data
    perc_acc, scores = [],[]
    for idx in xrange(0, val_x.shape[0], batch_size):
        feed_dict = {x: val_x[idx:idx+batch_size],
                     weights: 1.0/val_yerr[idx:idx+batch_size], is_training: 0}
        y_pred = sess.run(preds, feed_dict=feed_dict)
        #print y_pred.shape, val_y[idx:idx+batch_size].shape
        score = r2_score(val_y[idx:idx+batch_size], y_pred)
        scores.append(score)
        
        perc_acc = np.mean(emu._y_std*np.abs(val_y[idx:idx+batch_size]-y_pred)/np.abs(emu._y_std*val_y[idx:idx+batch_size] + emu._y_mean) )

        
    print 'Val score: %.6f, %.2f %% Loss'%(np.mean(np.array(scores)), 100*np.mean(np.array(perc_acc)))

In [249]:
from sklearn.model_selection import train_test_split

In [250]:
device = '/device:GPU:0'
#device = '/cpu:0'
def train_part(model_init_fn, optimizer_init_fn,num_params, train_data, val_data, hidden_sizes,\
               num_epochs=1, batch_size = 200, print_every=10):
    tf.reset_default_graph()    
    with tf.device(device):
        # Construct the computational graph we will use to train the model. We
        # use the model_init_fn to construct the model, declare placeholders for
        # the data and labels
        x = tf.placeholder(tf.float32, [None,num_params])
        y = tf.placeholder(tf.float32, [None])
        weights = tf.placeholder(tf.float32, [None])
        
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        preds = model_init_fn(x, hidden_sizes, is_training)

        # Compute the loss like we did in Part II
        #loss = tf.reduce_mean(loss)
        
    with tf.device('/cpu:0'):
        loss = tf.losses.mean_squared_error(labels=y, predictions=preds, weights = weights)#weights?

    with tf.device(device):
        optimizer = optimizer_init_fn()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)
        
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #t = 0
        train_x, train_y, train_yerr = train_data
        rand_idxs = range(train_x.shape[0])
        for epoch in range(num_epochs):
            #print('Starting epoch %d' % epoch)
            np.random.shuffle(rand_idxs)
            for idx in xrange(0, train_x.shape[0], batch_size):
                feed_dict = {x: train_x[rand_idxs[idx:idx+batch_size]],\
                             y: train_y[rand_idxs[idx:idx+batch_size]],\
                             weights: 1/train_yerr[rand_idxs[idx:idx+batch_size]],\
                             is_training:1}
                loss_np, _, preds_np = sess.run([loss, train_op, preds], feed_dict=feed_dict)
            if epoch % print_every == 0:
                print('Epoch %d, loss = %e' % (epoch, loss_np))
                check_accuracy(sess, val_data, batch_size, x, weights, preds, is_training=is_training)
            #t += 1

In [251]:
train_part(n_layer_fc, optimizer_init_fn, x_np.shape[1],\
           (x_train, y_train, yerr_train), (x_test, y_test, yerr_test),  [300, 200], num_epochs= int(1e4), batch_size = 1000, print_every = 100)

Epoch 0, loss = 2.629813e+02
Val score: -0.652878, 30.30 % Loss
Epoch 100, loss = 7.787210e+00
Val score: 0.512023, 42.14 % Loss
Epoch 200, loss = 6.124205e+00
Val score: 0.769851, 7.81 % Loss
Epoch 300, loss = 3.798334e+00
Val score: 0.778844, 7.52 % Loss
Epoch 400, loss = 1.395549e+00
Val score: 0.739999, 18.36 % Loss
Epoch 500, loss = 1.369787e+00
Val score: 0.687449, 25.80 % Loss
Epoch 600, loss = 4.806212e-01
Val score: 0.650724, 29.70 % Loss
Epoch 700, loss = 5.649343e-01
Val score: 0.643406, 30.10 % Loss
Epoch 800, loss = 5.112386e-01
Val score: 0.636618, 30.61 % Loss
Epoch 900, loss = 2.028414e-01
Val score: 0.635753, 30.38 % Loss
Epoch 1000, loss = 4.253392e-01
Val score: 0.634891, 30.31 % Loss


KeyboardInterrupt: 

In [ ]:
np.abs(emu.goodness_of_fit(training_file, statistic = 'log_frac')).mean()

In [ ]:
np.abs(emu.goodness_of_fit(training_file, statistic = 'frac')).mean()

In [ ]:
fit_idxs = np.argsort(gof.mean(axis = 1))

In [ ]:
emu.goodness_of_fit(training_file).mean()#, statistic = 'log_frac')).mean()

In [ ]:
model = emu._emulator

In [ ]:
ypred = model.predict(emu.x)

In [ ]:
plt.hist( np.log10( (emu._y_std+1e-5)*np.abs(ypred-emu.y)/np.abs((emu._y_std+1e-5)*emu.y+emu._y_mean) ))

In [ ]:
( (emu._y_std+1e-5)*np.abs(ypred-emu.y)/np.abs((emu._y_std+1e-5)*emu.y+emu._y_mean) ).mean()

In [ ]:
emu._y_mean, emu._y_std

In [ ]:
for idx in fit_idxs[:10]:
    print gof[idx].mean()
    print (ypred[idx*emu.n_bins:(idx+1)*emu.n_bins]-emu.y[idx*emu.n_bins:(idx+1)*emu.n_bins])/emu.y[idx*emu.n_bins:(idx+1)*emu.n_bins]
    plt.plot(emu.scale_bin_centers, ypred[idx*emu.n_bins:(idx+1)*emu.n_bins], label = 'Emu')
    plt.plot(emu.scale_bin_centers, emu.y[idx*emu.n_bins:(idx+1)*emu.n_bins], label = 'True')
    plt.legend(loc='best')
    plt.xscale('log')
    plt.show()

In [ ]:
print dict(zip(emu.get_param_names(), emu.x[8*emu.n_bins, :]*emu._x_std+emu._x_mean))

In [ ]:
emu.get_param_names()

In [ ]:
emu._ordered_params

In [ ]:
gof = emu.goodness_of_fit(training_file, statistic = 'frac')
print gof.mean()

In [ ]:
for row in gof:
    print row

In [ ]:
gof = emu.goodness_of_fit(training_file, statistic = 'frac')
print gof.mean()

In [ ]:
model = emu._emulator

In [ ]:
model.score(emu.x, emu.y)

In [ ]:
ypred = model.predict(emu.x)

np.mean(np.abs(ypred-emu.y)/emu.y)

In [ ]:
plt.plot(emu.scale_bin_centers, np.abs(gof.mean(axis = 0)) )
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.01)
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.05)
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.1)


plt.loglog();

In [ ]:
plt.plot(emu.scale_bin_centers, np.abs(gof.T),alpha = 0.1, color = 'b')
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.01, lw = 2, color = 'k')
plt.loglog();

In [ ]:
gof[:,i].shape

In [ ]:
for i in xrange(gof.shape[1]):
    plt.hist(np.log10(gof[:, i]), label = str(i), alpha = 0.2);
    
plt.legend(loc='best')
plt.show()